Code To Augment Images


In [ ]:
import os
import cv2
import albumentations as A

# --- Configuration ---
INPUT_DIR = "masked"
OUTPUT_DIR = "augmented_masked"
# We need ~33 augmentations per image to reach 300. Let's make it 34.
# Total images will be 9 * 34 = 306.
TARGET_COUNT_PER_IMAGE = 34

# --- Define the Augmentation Pipeline ---
# This pipeline applies a series of random transformations to each image.
transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.ShiftScaleRotate(shift_limit=0.06, scale_limit=0.1, rotate_limit=45, p=0.8),
    A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.7),
    A.GaussNoise(var_limit=(10.0, 50.0), p=0.5),
    A.MotionBlur(blur_limit=7, p=0.5)
])

# --- Main Script ---
def augment_images():
    """
    Reads images from INPUT_DIR, applies augmentations, and saves them to OUTPUT_DIR.
    """
    # Create output directory if it doesn't exist
    os.makedirs(OUTPUT_DIR, exist_ok=True)

    try:
        image_files = [f for f in os.listdir(INPUT_DIR) if f.endswith(('.png', '.jpg', '.jpeg'))]
        if not image_files:
            print(f"Error: No images found in the '{INPUT_DIR}' directory.")
            return
    except FileNotFoundError:
        print(f"Error: Input directory '{INPUT_DIR}' not found. Please create it.")
        return

    total_generated = 0
    print(f"Found {len(image_files)} source images. Starting augmentation...")

    # Loop through each source image
    for filename in image_files:
        image_path = os.path.join(INPUT_DIR, filename)
        # Read the image using OpenCV
        image = cv2.imread(image_path)
        # Convert from BGR (OpenCV default) to RGB for Albumentations
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        base_filename = os.path.splitext(filename)[0]
        print(f"  -> Augmenting '{filename}'...")

        # Generate the target number of augmented versions
        for i in range(TARGET_COUNT_PER_IMAGE):
            # Apply the augmentation pipeline
            augmented_data = transform(image=image)
            augmented_image = augmented_data['image']

            # Convert back to BGR to save with OpenCV
            augmented_image_bgr = cv2.cvtColor(augmented_image, cv2.COLOR_RGB2BGR)

            # Define the new filename and save the image
            output_filename = f"{base_filename}_aug_{i+1}.jpg"
            cv2.imwrite(os.path.join(OUTPUT_DIR, output_filename), augmented_image_bgr)
            total_generated += 1

    print(f"\nDone! ✅ Generated {total_generated} images in the '{OUTPUT_DIR}' folder.")

if __name__ == "__main__":
    augment_images()

c:\Users\sreeh\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\sreeh\AppData\Local\Programs\Python\Python310\lib\site-packages\albumentations\core\validation.py:114: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)
C:\Users\sreeh\AppData\Local\Temp\ipykernel_28412\1663660347.py:19: UserWarning: Argument(s) 'var_limit' are not valid for transform GaussNoise
  A.GaussNoise(var_limit=(10.0, 50.0), p=0.5),


Found 9 source images. Starting augmentation...
  -> Augmenting '1.jpg'...
  -> Augmenting '2.jpg'...
  -> Augmenting '3.jpg'...
  -> Augmenting '4.jpg'...
  -> Augmenting '5.jpg'...
  -> Augmenting '6.jpg'...
  -> Augmenting '7.jpg'...
  -> Augmenting 'home-vada.png'...
  -> Augmenting 'tst_vada.jpg'...

Done! ✅ Generated 306 images in the 'augmented_vadas' folder.


Splitting Dataset

In [ ]:
import os
import shutil
import random

# --- Configuration ---
# Source folders containing your full augmented dataset
SOURCE_IMAGES_DIR = "augmented_images"
SOURCE_MASKS_DIR = "augmented_masks"

# Destination folders to be created
DEST_TRAIN_A = "trainA"  # 80% of real images
DEST_TEST_A = "testA"    # 20% of real images
DEST_TRAIN_B = "trainB"  # 80% of masks
DEST_TEST_B = "testB"    # 20% of masks

# Split ratio
SPLIT_RATIO = 0.20

# --- Main Script ---
def create_final_dataset():
    """
    Copies files from source augmented folders into a new, split
    train/test structure required by the CycleGAN model.
    """
    # Create all destination directories
    for path in [DEST_TRAIN_A, DEST_TEST_A, DEST_TRAIN_B, DEST_TEST_B]:
        os.makedirs(path, exist_ok=True)

    # Get the master list of filenames from the images folder
    try:
        filenames = os.listdir(SOURCE_IMAGES_DIR)
    except FileNotFoundError:
        print(f"Error: Source directory '{SOURCE_IMAGES_DIR}' not found.")
        return

    # Shuffle the list to ensure a random split
    random.shuffle(filenames)

    # Calculate the split point
    split_index = int(len(filenames) * (1 - SPLIT_RATIO))

    # Divide the list into training and testing sets
    train_files = filenames[:split_index]
    test_files = filenames[split_index:]

    print("Splitting files...")

    # Copy training files
    for filename in train_files:
        # Copy image to trainA
        shutil.copy2(os.path.join(SOURCE_IMAGES_DIR, filename), os.path.join(DEST_TRAIN_A, filename))
        # Copy corresponding mask to trainB
        shutil.copy2(os.path.join(SOURCE_MASKS_DIR, filename), os.path.join(DEST_TRAIN_B, filename))

    # Copy testing files
    for filename in test_files:
        # Copy image to testA
        shutil.copy2(os.path.join(SOURCE_IMAGES_DIR, filename), os.path.join(DEST_TEST_A, filename))
        # Copy corresponding mask to testB
        shutil.copy2(os.path.join(SOURCE_MASKS_DIR, filename), os.path.join(DEST_TEST_B, filename))

    print("\nDone! ✅")
    print(f"Total training images (trainA): {len(os.listdir(DEST_TRAIN_A))}")
    print(f"Total testing images (testA):   {len(os.listdir(DEST_TEST_A))}")
    print(f"Total training masks (trainB):  {len(os.listdir(DEST_TRAIN_B))}")
    print(f"Total testing masks (testB):    {len(os.listdir(DEST_TEST_B))}")

if __name__ == "__main__":
    create_final_dataset()